In [56]:
import numpy as np
import hashlib
import random
import secrets

In [ ]:
# Helper functions 

# Returns sha256 hash of the data  
def hash_data(data):
    return hashlib.sha256(str(data).encode('utf-8')).hexdigest()

# builds a merkle tree with an arbitrary string salt, in this case "ETH"
def build_merkle_tree_eth(leaves):
    # Salt leaves with "eth", the salt should be secret to the prover
    current_layer = [hash_data(str(item) + "eth") for item in leaves]
    tree = [current_layer]
    while len(current_layer) > 1:
        next_layer = []
        for i in range(0, len(current_layer), 2):
            left = current_layer[i]
            right = current_layer[i + 1] if i + 1 < len(current_layer) else left 
            next_layer.append(hash_data(left + right))
        current_layer = next_layer
        tree.append(current_layer)
    return tree

# verifies a merkle proof using a tree at an index, returns the complete path
def get_merkle_proof(tree, index):
    path = []
    current_idx = index
    for layer_idx in range(len(tree) - 1):
        layer = tree[layer_idx]
        is_left = (current_idx % 2 == 0)
        sibling_idx = current_idx + 1 if is_left else current_idx - 1
        if sibling_idx >= len(layer): sibling_idx = current_idx # Handle duplicate
        
        path.append({
            'sibling_hash': layer[sibling_idx],
            'direction': 'right' if is_left else 'left'
        })
        current_idx //= 2
    return path

# verifies a merkle proof given the leaf, path and expected root
def verify_merkle_proof(leaf_value, proof, expected_root):
    current_hash = hash_data(str(leaf_value) + "eth")
    for step in proof:
        if step['direction'] == 'right':
            current_hash = hash_data(current_hash + step['sibling_hash'])
        else:
            current_hash = hash_data(step['sibling_hash'] + current_hash)
    return current_hash == expected_root

# used for fiat-shamir heuristic, hashes the root with the public input
def get_fiat_shamir_challenge(merkle_root, public_array):
    challenge_hash = hashlib.sha256((str(merkle_root) + str(public_array)).encode('utf-8')).hexdigest()
    return int(challenge_hash, 16) % len(public_array)


In [116]:
# prover from the example in Mastering Ethereum 2nd edition chapted 17
S = np.array([1, 1, 5, 7])
a = np.array([1, 1, 1, -1]) 
r = random.randint(1, 1000) 
coin = random.choice([1, -1])

# Calculate Witness (cumulative sum)
coin_a = a * coin
new_w = np.cumsum(S * coin_a) + r 

# Generate Commitment
merkle_tree = build_merkle_tree_eth(new_w)
merkle_root = merkle_tree[-1][0]
print("root:", merkle_root)

# Generate Fiat-Shamir challenge
verifier_j = get_fiat_shamir_challenge(merkle_root, S)
print("Fiat-Shamir Challenge:", verifier_j)

# To prove S[j], we need the difference between w[j] and w[j-1].
# use modulo to handle the wrap-around case (if j=0, we need w[-1]) instead of rolling.
idx_1 = (verifier_j - 1) % len(S)
idx_2 = verifier_j 

val_1 = new_w[idx_1]
val_2 = new_w[idx_2]

proof_1 = get_merkle_proof(merkle_tree, idx_1)
proof_2 = get_merkle_proof(merkle_tree, idx_2)


# Verifier

# Re-calculate challenge to ensure Prover didn't cheat the index
calculated_j = get_fiat_shamir_challenge(merkle_root, S)

if calculated_j != verifier_j:
    print("FAILURE: Prover used the wrong index!")
else:
    # Verify proofs
    check_1 = verify_merkle_proof(val_1, proof_1, merkle_root)
    check_2 = verify_merkle_proof(val_2, proof_2, merkle_root)

    if not (check_1 and check_2):
        print("FAILURE: Invalid Merkle Proofs.")
    else:
        # Verify required identity |s[i]| = |w[i + 1] – w[i]|
        calculated_diff = abs(val_2 - val_1)
        expected_diff = abs(S[verifier_j])

        print(f"Checking Index {verifier_j}: |{val_2} - {val_1}| == {expected_diff}?")

        if calculated_diff == expected_diff:
            print("Proof Vaild")
        else:
            print("Proof Invalid")

root: 937f48c55cb012f0e9e8645cffeb53dba02d78381de1ef16c818deb5b1da5309
Fiat-Shamir Challenge: 2
Checking Index 2: |184 - 189| == 5?
Proof Vaild


1. Prover Commitment: 24dda7d5abf10fe8d16a699ef5b7f00f0f1e6843911284b9737c81a28dac6c14
2. Fiat-Shamir Derived Index to Prove: 3

--- NON-INTERACTIVE VERIFIER STARTING ---
Checking Index 3: |76 - 83| == 7?
SUCCESS: Math assertion holds. Proof Accepted.
